<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
OPTIONS_MODEL_PATH =  os.path.join(NOMEROFF_NET_DIR, "models/numberplate_options_2019_05_20.h5")

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.pb
mode = "cpu"
OCR_NP_UKR_TEXT = os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ua_12-{}.h5".format(mode))
OCR_NP_EU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_eu_2-{}.h5".format(mode))
OCR_NP_RU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ru_3-{}.h5".format(mode))
OCR_NP_KZ_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_kz_4-{}.h5".format(mode))

sys.path.append(NOMEROFF_NET_DIR)

Also you may use .pb models for options detectors or OCR detectors. 
More about conversion and preparing for inference withit https://github.com/ria-com/nomeroff-net/blob/master/examples/inference_convert.ipynb.

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector({
    "class_region": ["xx_unknown", "eu_ua_2015", "eu_ua_2004", "eu_ua_1995", "eu", "xx_transit", "ru", "kz"]
})
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    },
    "ru": {
        "for_regions": ["ru"],
        "model_path": OCR_NP_RU_TEXT
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": OCR_NP_KZ_TEXT
    }
})

Using TensorFlow backend.
W0528 16:15:51.835785 139830092576064 deprecation.py:237] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0528 16:15:54.224163 139830092576064 deprecation.py:237] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.

W0528 16:15:54.235183 139830092576064 deprecation.py:506] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:423: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_ind is deprecated and will be removed in a future version.
Instructions for updating:
box_ind is deprecated, use box_indices instead
W0528 16:15:54.416700 139830092576064 deprecation.py:237] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:721: The name tf.sets.set_intersection is deprecated. Please use tf.sets.intersection instead.

W0528 16:15:54.4189

In [3]:
# # Walking through the ./examples/images/ directory and checking each of the images for license plates.
# rootDir = '/var/www/datasets/fake-region/dnr/'

# max_img_w = 1600
# for dirName, subdirList, fileList in os.walk(rootDir):
#     for fname in fileList:
#         baseName = os.path.splitext(os.path.basename(fname))[0]
#         img_path = os.path.join(dirName, fname)
#         print(img_path)
#         img = mpimg.imread(img_path)
#         #plt.axis("off")
#         #plt.imshow(img)
#         #plt.show()
         
#         # corect size for better speed
#         img_w = img.shape[1]
#         img_h = img.shape[0]
#         img_w_r = 1
#         img_h_r = 1
#         if img_w > max_img_w:
#             resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
#             img_w_r = img_w/max_img_w
#             img_h_r = img_h/(max_img_w/img_w*img_h)
#         else:
#             resized_img = img
        
#         try:
#             NP = nnet.detect([resized_img]) 

#             # Generate image mask.
#             cv_img_masks = await filters.cv_img_mask_async(NP)

#             # Detect points.
#             arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
#             #print(arrPoints)
#             arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
#             arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r

#             # cut zones
#             zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
#             toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

#             # find standart
#             regionIds, stateIds, countLines = optionsDetector.predict(zones)
#             regionNames = optionsDetector.getRegionLabels(regionIds)
#             foundNumber=0
#             for zone, points, regionId in zip(toShowZones, arrPoints, regionIds):
#                 #plt.axis("off")
#                     mpimg.imsave("/var/www/datasets/fake-region/dnr/img/{}_{}.png".format(baseName, foundNumber), zone)
#                     foundNumber += 1
#                     #plt.imshow(zone)
#                 #plt.show()


#             #print(regionNames)

#             # find text with postprocessing by standart  
#             #textArr = textDetector.predict(zones, regionNames, countLines)
#             #textArr = await textPostprocessingAsync(textArr, regionNames)
#             #print(textArr)
#         except Exception as e:
#             print("Error")
#             print(e)

In [4]:
# Walking through the ./examples/images/ directory and checking each of the images for license plates.
rootDir = '/var/www/datasets/kg'

max_img_w = 1600
i = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        baseName = os.path.splitext(os.path.basename(fname))[0]
        img_path = os.path.join(dirName, fname)
        print(i, img_path)
        i += 1
        img = mpimg.imread(img_path)
        #plt.axis("off")
        #plt.imshow(img)
        #plt.show()
         
        # corect size for better speed
        img_w = img.shape[1]
        img_h = img.shape[0]
        img_w_r = 1
        img_h_r = 1
        if img_w > max_img_w:
            resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
            img_w_r = img_w/max_img_w
            img_h_r = img_h/(max_img_w/img_w*img_h)
        else:
            resized_img = img
        
        try:
            NP = nnet.detect([resized_img]) 

            # Generate image mask.
            cv_img_masks = await filters.cv_img_mask_async(NP)

            # Detect points.
            arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
            #print(arrPoints)
            arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
            arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r

            # cut zones
            zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
            toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

            # find standart
            #regionIds, stateIds, countLines = optionsDetector.predict(zones)
            #regionNames = optionsDetector.getRegionLabels(regionIds)
            foundNumber=0
            for zone, points in zip(toShowZones, arrPoints):
                #plt.axis("off")
                    mpimg.imsave("/var/www/nomeroff-net/datasets/kg/img/{}_{}.png".format(baseName, foundNumber), zone)
                    foundNumber += 1
                    #plt.imshow(zone)
                #plt.show()


            #print(regionNames)

            # find text with postprocessing by standart  
            #textArr = textDetector.predict(zones, regionNames, countLines)
            #textArr = await textPostprocessingAsync(textArr, regionNames)
            #print(textArr)
        except Exception as e:
            print("Error")
            print(e)

0 /var/www/datasets/kg/3bsYM9m7-b96691fe0ab6f78e6335b525f6ba36ea-medium.jpg
1 /var/www/datasets/kg/H2pFL4L4-095886913600abc7435f0ffc1b8bb8f8-medium.jpg
2 /var/www/datasets/kg/bXnnge0j-1168e865f4469fce672a40405c2f357e-medium.jpg
3 /var/www/datasets/kg/Q4tslF7t-c3be06b7e27683e311ba9aaed60c0ce3-medium.jpg
4 /var/www/datasets/kg/Q82xRFjS-e76c33f36f27422d9c5c90782482ad44-medium.jpg
5 /var/www/datasets/kg/DAWKQqbj-1bd7e23906a947a581a0e661991a8926-medium.jpg
6 /var/www/datasets/kg/16rUUlUe-64f5d108daf39d532eb84c967512172c-medium.jpg
7 /var/www/datasets/kg/4o0zgYL3-0cd8fa17cdfdae9c5b6c2bc8db9b26c3-medium.jpg
8 /var/www/datasets/kg/CmwHwvmr-68f0f0d05c820359f0601c8554a108b4-medium.jpg
9 /var/www/datasets/kg/EYhquQQh-70994f4038d2a02815ac0afa785b7c4b-medium.jpg
10 /var/www/datasets/kg/6ZVvxkI9-fdb284b598b715e705de60ac98ad7426-medium.jpg
11 /var/www/datasets/kg/xbOvNeza-6012133857a6da5749efe3f271f68cc3-medium.jpg
12 /var/www/datasets/kg/WxvQp4WO-c06027a3e398de5402e414d4ff0dab03-medium.jpg
13 /var/w

107 /var/www/datasets/kg/Rrqvwzug-efa96164abb7834aa552f74eae211753-medium.jpg
108 /var/www/datasets/kg/gkQGfVws-2931acd45a66572447fabd12bd9f5de0-medium.jpg
109 /var/www/datasets/kg/Yw4Jn1og-6e596b48cef8ad7a31086e3cbee90770-medium.jpg
110 /var/www/datasets/kg/Hnl3kq4T-abf18ae8fc8c6063ec35de19dea9d91e-medium.jpg
111 /var/www/datasets/kg/ws633oEx-67b88cc5447d6b636394fe0c0391bf4a-medium.jpg
112 /var/www/datasets/kg/fyWkiY0i-bab4f92014170df9df0282a5fa7df089-medium.jpg
113 /var/www/datasets/kg/JfvEa5d5-50f8dfe5c0c68c6ace0afcdc997355a3-medium.jpg
114 /var/www/datasets/kg/xbOvNeza-5d6ef51f5e599e70550b67674520ad07-medium.jpg
115 /var/www/datasets/kg/IgNx2kXK-42165ebccb17b553581d513d1b514edd-medium.jpg
116 /var/www/datasets/kg/xq91cS39-2d15237d85d74747eb60766cf05d3b74-medium.jpg
117 /var/www/datasets/kg/YNW4nVj5-a2b57066843d56b5d65697f179a96c48-medium.jpg
118 /var/www/datasets/kg/pMJJM92v-9e68d1bdc0c4baee2ce74ec57abd4c02-medium.jpg
119 /var/www/datasets/kg/u26OETtm-02eaf5d5c6c24afbaf2181ba53648a

213 /var/www/datasets/kg/6ZVvxkI9-a9a631b1dd9be75158f79581c4279918-medium.jpg
214 /var/www/datasets/kg/pKqq2nWn-acc351500505900658dd5642649f8de2-medium.jpg
215 /var/www/datasets/kg/FRvohmXW-3a7a95a82690bd49d3556d05e5744c5e-medium.jpg
216 /var/www/datasets/kg/SNajyJPl-690d6e9ae12cee1e2712b801f8bca83c-medium.jpg
217 /var/www/datasets/kg/vywSuv7z-d05401658180ead3465c8b8b26d5bedb-medium.jpg
218 /var/www/datasets/kg/dzwwamn7-0b2255914b38edb7d2f3a3bbd487428a-medium.jpg
219 /var/www/datasets/kg/I3Fj2L3H-911dff0e1f84bb5f9784093c6d1e6636-medium.jpg
220 /var/www/datasets/kg/JCfHXz23-8ecff8235accbc732b458624f5a0255d-medium.jpg
221 /var/www/datasets/kg/3beQjKWa-53f7308d8e16fc81897f6bce57bb968c-medium.jpg
222 /var/www/datasets/kg/T3M3Oblw-4ab1ba8c359d8f872db6e4e82e8509f4-medium.jpg
223 /var/www/datasets/kg/lKv3rUzH-877aeb6e75ed116b51c43a775c201ab1-medium.jpg
224 /var/www/datasets/kg/WyGEsHVD-975a15952d39c2a9fd8fbe00246747de-medium.jpg
225 /var/www/datasets/kg/f5qly11l-ee599bf4102dbe074c76ce8f237da6

319 /var/www/datasets/kg/MonSICW9-edcc0a64b68e3366a839817de4dfd15d-medium.jpg
320 /var/www/datasets/kg/rbgpQ9hE-c275536b3765b98bab5dbfcdfdd88112-medium.jpg
321 /var/www/datasets/kg/nb0xaRZM-312598cc0c86d81968c95b936a7fee73-medium.jpg
322 /var/www/datasets/kg/HGGcSb2j-9a8b1d4732da79f699f81970bbedc1fb-medium.jpg
323 /var/www/datasets/kg/D1S60aqB-10f42fda3a0a40cf8fe71ddc9167463a-medium.jpg
324 /var/www/datasets/kg/2kjdhGSc-1f001ad19954d07dd3d399495e0bbabb-medium.jpg
325 /var/www/datasets/kg/gvjFGG25-7cd7f5bcd219050c38b2f8f349567166-medium.jpg
326 /var/www/datasets/kg/g20IT01V-957d3fbd431d717f3a84847d34fa61e7-medium.jpg
327 /var/www/datasets/kg/d67qyd2D-8c130c424f2cb05f9d4719a7e115f0af-medium.jpg
328 /var/www/datasets/kg/OCv5ZxRV-2c5f58ce02333ebc773b683f557c36e9-medium.jpg
329 /var/www/datasets/kg/8fbNHmfG-8402663b3f1a306600200ab400325c26-medium.jpg
330 /var/www/datasets/kg/11SHEV5I-50dac1a0a9720ac484fd0ba79151e239-medium.jpg
331 /var/www/datasets/kg/5bJZL4VY-c5b9e16db98d1be2c871424df90bb9

425 /var/www/datasets/kg/uB1ZlnWd-29ca8d53ae8805276370954e65beb781-medium.jpg
426 /var/www/datasets/kg/HiKQju0x-2dfe4c3372949251f0125dfce4850b88-medium.jpg
427 /var/www/datasets/kg/0yeV7Gnx-840ba2b67daf8cec255247a73d961ef6-medium.jpg
428 /var/www/datasets/kg/2sBGcHiJ-f5e0eed5fea1771ad78b2fc821a47495-medium.jpg
429 /var/www/datasets/kg/t7QaFycT-48deb3d4c3556c95e0e316e7afb61527-medium.jpg
430 /var/www/datasets/kg/H7JnHa1Z-e03756fbc6922a21ac2fd3458fe1a0c9-medium.jpg
431 /var/www/datasets/kg/UBBo8rPU-2bdeb03a14af405903be945b9bc73cee-medium.jpg
432 /var/www/datasets/kg/zwqXap8Y-d7b64c914e2d2ea83344f62b559063fc-medium.jpg
433 /var/www/datasets/kg/uclSWKjR-8c3bb6400ee5c99e3c74b71b840662ea-medium.jpg
434 /var/www/datasets/kg/GUpNoOGl-4f40d3c692711efcdac83818e0b69ae9-medium.jpg
435 /var/www/datasets/kg/2CPPbaja-8db8a8a9c04a89896f04c11957193413-medium.jpg
436 /var/www/datasets/kg/3bsYM9m7-092ca4d5bf2630f67d0c0622bed413a0-medium.jpg
437 /var/www/datasets/kg/So6xCOPA-91246bd00bbd05f63bdf4805076b74

...

5738 /var/www/datasets/kg/HvH2NaXT-7bc03f8b908cf394f3a152ce5235af3a-medium.jpg
5739 /var/www/datasets/kg/c367JTWS-ea291b7ae86fbd0227cfc2bfc0ac2393-medium.jpg
5740 /var/www/datasets/kg/JNw5BXsP-b791e4c96b9672c05497618e3f54eae8-medium.jpg
5741 /var/www/datasets/kg/hRo3Xij5-98f0ef820772147383f2d1bb50e93bb5-medium.jpg
5742 /var/www/datasets/kg/JfvEa5d5-d189d7a7275b17d1669b5d5d6eb0370c-medium.jpg
5743 /var/www/datasets/kg/DhOJ1Y5q-d501b93b87806134799368b1fb961b97-medium.jpg
5744 /var/www/datasets/kg/AprC6UEj-38558aa9551db05d5de527f28d50fb9f-medium.jpg
5745 /var/www/datasets/kg/0fRsgjcc-40d2a700c9bf84a1984f24c070959d4a-medium.jpg
5746 /var/www/datasets/kg/X4FMNI4L-f35770c84e82afbec22b0f13388b1773-medium.jpg
5747 /var/www/datasets/kg/igVl8czy-3e2c8ea9f56ef5fa1cd7c7850ae190c3-medium.jpg
5748 /var/www/datasets/kg/g20IT01V-e4a271ba026966ffcb0c43915ede2b12-medium.jpg
5749 /var/www/datasets/kg/c2taALGP-e41ec652bb0f018372a6b7cad722931d-medium.jpg
5750 /var/www/datasets/kg/zH31YvDy-12a1cfe31a61a5cbb

5842 /var/www/datasets/kg/rQWs6S3S-bc6e0a73c695c8e0f762851bad1cad7b-medium.jpg
5843 /var/www/datasets/kg/JKKRBPMd-3a53551343b0109c59e22df04d9f0951-medium.jpg
5844 /var/www/datasets/kg/wbdNyyaI-a22c0e77499709c4dfe2b5c685857ba3-medium.jpg
5845 /var/www/datasets/kg/0dufaZfD-331b7b585488e54172fb8bf7391c6ce4-medium.jpg
5846 /var/www/datasets/kg/t8Q1leZH-11543ac899a97166d067e0af43d252d1-medium.jpg
5847 /var/www/datasets/kg/rZDGouRW-d44cd8132c002ac4fa0e9e6c24e2dd7c-medium.jpg
5848 /var/www/datasets/kg/UpQyNpRe-d64993d3f6f58d0bb46197a6f0ef2c1c-medium.jpg
5849 /var/www/datasets/kg/eGU8D6Ys-84299450b017137ba3496d3f2f0d37c9-medium.jpg
5850 /var/www/datasets/kg/pWA8Pt39-60879e77e1b27c3acc35247510c133b9-medium.jpg
5851 /var/www/datasets/kg/sJbKxM3p-3de41937925c1743010904312ff5d77a-medium.jpg
5852 /var/www/datasets/kg/JNAv2xNG-94e27428a23df6c3a8d814012c9a8ad2-medium.jpg
5853 /var/www/datasets/kg/QuYvebBO-272d33d5a9aa82c7e3ffa207ad1d0207-medium.jpg
5854 /var/www/datasets/kg/jITHuice-16c0d444c2dd082c8

5946 /var/www/datasets/kg/MUCLl6kA-8bcc1d790c4eadfa7c1ab0ffbc1a6bdf-medium.jpg
5947 /var/www/datasets/kg/F362TipL-061ffc5ca6d5ea727aafa1d829733da4-medium.jpg
5948 /var/www/datasets/kg/8YgogxFu-9af5ee19c84ee721cdab092d60c63acf-medium.jpg
5949 /var/www/datasets/kg/vdN9ZAZU-61d0807c3bdd4105f139d135fa3269c4-medium.jpg
5950 /var/www/datasets/kg/5kqsyLVE-57f9a61c3a52bb02c952b4ac72c41e6c-medium.jpg
5951 /var/www/datasets/kg/vSvRxsea-cc3c79c093b3233a8faa509534ab0ad1-medium.jpg
5952 /var/www/datasets/kg/TxNKmfLX-b938a269fb84ebf280d3168f81fbcf6c-medium.jpg
5953 /var/www/datasets/kg/3igitRqw-71ec7b64ee11e8e6a8e5c44455919859-medium.jpg
5954 /var/www/datasets/kg/rbgpQ9hE-f2b8f93fd11e1cad6595e3a99855db2d-medium.jpg
5955 /var/www/datasets/kg/dtUy43y6-83ae23bfdac628c8d994ffe7d2cce9ce-medium.jpg
5956 /var/www/datasets/kg/owAzIFb0-2d4aa74c245aed708d38c94ca5f9e814-medium.jpg
5957 /var/www/datasets/kg/VhkUItcC-fbe81e665148041a2c29d74ba58d4bf3-medium.jpg
5958 /var/www/datasets/kg/MZUX0rMq-d95b840080d2f6346

6050 /var/www/datasets/kg/o0pQqhAj-da8b6795014284c8cfd51a8d21fca53c-medium.jpg
6051 /var/www/datasets/kg/IV4SqAez-6f1e30b3ad594c0b4ff42f11163a236a-medium.jpg
6052 /var/www/datasets/kg/6FvIz3eR-4269cbd4fbba7ed9c016561d6bb0f16d-medium.jpg
6053 /var/www/datasets/kg/nWlFbfJC-db596827f8884a5bb00dddd539f60666-medium.jpg
6054 /var/www/datasets/kg/3GZkjyIW-e8fff6303a2fef678de139bd92e535b6-medium.jpg
6055 /var/www/datasets/kg/RZtYaxGi-fd60e909b1107c56ba2b07d89893056c-medium.jpg
6056 /var/www/datasets/kg/wIMlrK2Y-8de119881ae91eab39049a1443c22d27-medium.jpg
6057 /var/www/datasets/kg/CCmupo7S-a80ccf8c520d87f7f1efcd237b4f27ad-medium.jpg
6058 /var/www/datasets/kg/U1e1CkY8-2e521963c155a19f819384bd98d06960-medium.jpg
6059 /var/www/datasets/kg/4raN82N3-f847725e1969a466fce3fe22c06062d7-medium.jpg
6060 /var/www/datasets/kg/NRtjaxYQ-6e7c9087f50c5539528263a77f037bc4-medium.jpg
6061 /var/www/datasets/kg/ZaloK5l7-eafccd8037927aec157591b5df635bbb-medium.jpg
6062 /var/www/datasets/kg/JCfHXz23-f3fffeb393d8a6f9d

6154 /var/www/datasets/kg/jsyeadBq-8838dfd0a5dbd9945d31240ab4c41716-medium.jpg
6155 /var/www/datasets/kg/8epvAeLi-f88eacdaa2974c2a68864844741587b2-medium.jpg
6156 /var/www/datasets/kg/BPPEt6Mh-99591058ee65e0fb12599f07bbd233c4-medium.jpg
6157 /var/www/datasets/kg/wzyrTYQl-b4a51222f200c3aa36a2041cf3d4cf23-medium.jpg
6158 /var/www/datasets/kg/ZFdniGZH-a053f390e07a8b2ede857c8f6cf118f1-medium.jpg
6159 /var/www/datasets/kg/fBO9pvsL-36ac60d1bc4d688ae9b0844fdc52010d-medium.jpg
6160 /var/www/datasets/kg/AeRrICxr-0c050c59acde846b9e91407dceaf44f7-medium.jpg
6161 /var/www/datasets/kg/1ziSkM12-baf9c91583c9bc56d7d58c2f5379cb7f-medium.jpg
6162 /var/www/datasets/kg/pWA8Pt39-9fb8f0e8676e266e8d84f9cef181fa46-medium.jpg
6163 /var/www/datasets/kg/vZO1T11v-646ae03c6637625a0cb8d657b544a351-medium.jpg
6164 /var/www/datasets/kg/KM3Wc2bM-b6e053871188ac4e9d6f491baefbb0f4-medium.jpg
6165 /var/www/datasets/kg/3fZ1bdIy-858ea294e803562cbc319c2a34baf2c7-medium.jpg
6166 /var/www/datasets/kg/ulIhvzla-4f1c8367d8d5fb2cc